# Proyecto
______________

API: https://dev.adalab.es/cinema/
Endpoint : https://dev.adalab.es/api/cinema/movies?year=2010&genre=Drama

- **Librerías y display**

In [1]:
#Importar librerías para su posterior uso en el código
import tqdm
from bs4 import BeautifulSoup
import requests
import pandas as pd
import re
import datetime
import json
from time import sleep
pd.set_option('display.max_columns', None) # para poder visualizar todas las columnas de los DataFrames
# Importar librerías de Selenium
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select

# Importar librería para la conexión con MySQL
# -----------------------------------------------------------------------
import mysql.connector
from mysql.connector import errorcode


# Importar librerías para manipulación y análisis de datos
# -----------------------------------------------------------------------
import pandas as pd
import numpy as np

# Comprobando que el display sea el máximo para mostrar mejor el data frame
pd.set_option('display.max_rows', 100)  # Cambia el número máximo de filas mostradas
pd.set_option('display.max_columns', 10)  # Cambia el número máximo de columnas mostradas
pd.set_option('display.width', 1000)  # Cambia el ancho máximo de la pantalla

- **Extracción API** (generos y rango años 15)

In [2]:

# Lista de años buscados: 15 años
anio_lista = list(range(2009, 2025))

# Lista de géneros
generos_lista = ["Drama", "Comedy", "Action", "Fantasy", "Horror", "Mystery", "Romance", "Thriller"]

# Lista vacía en la cual almacenaremos los datos extrañidos de la API
datos_peliculas = []

# Bucle para iterar por año y género
for anio in anio_lista:
    for genero in generos_lista:
        # Determinar URL
        url_movies = f"https://dev.adalab.es/api/cinema/movies?year={anio}&genre={genero}"
        
        # Llamada a API
        res_url_movies = requests.get(url_movies)
        
        # Pausa para la API
        sleep(1) 
        
        # Verificar la espuesta de la API
        if res_url_movies.status_code == 200:
            try:
                json_url_general = res_url_movies.json()
                 
                # Imprimir jason para verlo
                print(f"Response for {anio} - {genero}: {json_url_general}")
                
                # Verificar si la llamada tiene la info correcta
                if isinstance(json_url_general, dict) and 'results' in json_url_general:
                    movies = json_url_general['results']
                    
                    # Verificar que es lista
                    if isinstance(movies, list):
                        # Bucle por película para extraer info
                        for pelicula in movies:
                            id = pelicula.get("idOwner")
                            titulo = pelicula.get("title")
                            tipo = pelicula.get("type")
                            anio = pelicula.get("year")
                            genero = pelicula.get("genre")
                            
                            # Añadir info a la lista anteriormente vacía
                            datos_peliculas.append((id, titulo, tipo, anio, genero))
                    else:
                        print("Expected 'results' to be a list.")
                else:
                    print("Expected JSON structure not found.")
            except json.JSONDecodeError:
                print("Error decoding JSON response.")
        else:
            print(f"Error: {res_url_movies.status_code} - {res_url_movies.reason}")

# Convertir a pandas frame
df_peliculas = pd.DataFrame(datos_peliculas)

# Salvar la busqueda en un documento con formato CSV
df_peliculas.to_csv("movies_last_15_years.csv", index=False)

print("Data extraction complete and saved to movies_last_15_years.csv.")


Response for 2009 - Drama: {'info': {'count': 10}, 'results': [{'id': 59404, 'title': 'Apaföld', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0143558', 'image': 'https://m.media-amazon.com/images/M/MV5BYjA5Y2M3ZDAtNjg4OC00ZDQ5LWI4MzgtM2E0NTdjZmUwZjE1XkEyXkFqcGdeQXVyNDYzMDMxOTc@._V1_.jpg'}, {'id': 59405, 'title': 'Rózsaszín sajt', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0153140', 'image': 'https://m.media-amazon.com/images/M/MV5BMjE2NDAxNDE0Nl5BMl5BanBnXkFtZTgwMTU1MjQwNjE@._V1_.jpg'}, {'id': 59406, 'title': 'Sanam Teri Kasam', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0205380', 'image': 'https://m.media-amazon.com/images/M/MV5BYjc3MGNhMmItMjljOS00NjY5LThlMmQtYWM1NWZiOGJlY2EyXkEyXkFqcGdeQXVyMjU4NDY1ODA@._V1_.jpg'}, {'id': 59407, 'title': 'Twice as Dead', 'type': 'Movie', 'year': '2009', 'genre': 'Drama', 'idOwner': 'tt0254832', 'image': 'https://m.media-amazon.com/images/M/MV5BMTI2NjA5NDkyMl5BMl5BanBnXkFtZTcwNjAxNTEwMw@@

- Verificar el output

In [3]:
print(datos_peliculas)
print(type(datos_peliculas))

[('tt0143558', 'Apaföld', 'Movie', '2009', 'Drama'), ('tt0153140', 'Rózsaszín sajt', 'Movie', '2009', 'Drama'), ('tt0205380', 'Sanam Teri Kasam', 'Movie', '2009', 'Drama'), ('tt0254832', 'Twice as Dead', 'Movie', '2009', 'Drama'), ('tt0311698', 'Railed', 'Movie', '2009', 'Drama'), ('tt0327597', 'Coraline', 'Movie', '2009', 'Drama'), ('tt0344074', 'Mitsein', 'Movie', '2009', 'Drama'), ('tt0346631', 'Blood and Bone', 'Movie', '2009', 'Drama'), ('tt0361748', 'Inglourious Basterds', 'Movie', '2009', 'Drama'), ('tt0362478', 'The Box', 'Movie', '2009', 'Drama'), ('tt0067230', 'I Miss Sonia Henie', 'Short', '2009', 'Comedy'), ('tt0288190', 'El reino de los cielos', 'Movie', '2009', 'Comedy'), ('tt0304876', 'Unter Strom', 'Movie', '2009', 'Comedy'), ('tt0312958', 'Chinango', 'Movie', '2009', 'Comedy'), ('tt0345683', 'Mutant Swinger from Mars', 'Video', '2009', 'Comedy'), ('tt0353765', 'Los muertos van deprisa', 'Movie', '2009', 'Comedy'), ('tt0365873', 'Two-Minute Heist', 'Movie', '2009', 'Com

Sacar variable de solo movies

In [5]:
movie = [pelicula for pelicula in datos_peliculas if pelicula[2] == "Movie"]
print(movie)

[('tt0143558', 'Apaföld', 'Movie', '2009', 'Drama'), ('tt0153140', 'Rózsaszín sajt', 'Movie', '2009', 'Drama'), ('tt0205380', 'Sanam Teri Kasam', 'Movie', '2009', 'Drama'), ('tt0254832', 'Twice as Dead', 'Movie', '2009', 'Drama'), ('tt0311698', 'Railed', 'Movie', '2009', 'Drama'), ('tt0327597', 'Coraline', 'Movie', '2009', 'Drama'), ('tt0344074', 'Mitsein', 'Movie', '2009', 'Drama'), ('tt0346631', 'Blood and Bone', 'Movie', '2009', 'Drama'), ('tt0361748', 'Inglourious Basterds', 'Movie', '2009', 'Drama'), ('tt0362478', 'The Box', 'Movie', '2009', 'Drama'), ('tt0288190', 'El reino de los cielos', 'Movie', '2009', 'Comedy'), ('tt0304876', 'Unter Strom', 'Movie', '2009', 'Comedy'), ('tt0312958', 'Chinango', 'Movie', '2009', 'Comedy'), ('tt0353765', 'Los muertos van deprisa', 'Movie', '2009', 'Comedy'), ('tt0365873', 'Two-Minute Heist', 'Movie', '2009', 'Comedy'), ('tt0375568', 'Astro Boy', 'Movie', '2009', 'Comedy'), ('tt0365929', 'Whiteout', 'Movie', '2009', 'Action'), ('tt0373340', 'Ste

- Conectar con SQL, crear cursor y creación BBDD

In [6]:
# Conectar a SQL
cnx = mysql.connector.connect(user='root', password='AlumnaAdalab',
                              host='127.0.0.1')
# Crear cursor
mycursor = cnx.cursor()

# Crear base de datos
try:
    mycursor.execute("CREATE DATABASE BBDD_CinemExtract")
    print(mycursor)
except mysql.connector.Error as err: # verificar si hay errores
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1007 (HY000): Can't create database 'bbdd_cinemextract'; database exists
Error Code: 1007
SQLSTATE HY000
Message Can't create database 'bbdd_cinemextract'; database exists


- Crear tabla y columnas tabla 1 (**datos_peliculas**) ---- API

In [23]:
# Seleccionar la base de datos
try:
    mycursor.execute("USE BBDD_CinemExtract")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 1 = datos_peliculas, sus columnas y sus tipos de datos
try:
    mycursor.execute("""
    CREATE TABLE datos_peliculas (
        id_pelicula VARCHAR(15) PRIMARY KEY, 
        titulo_pelicula VARCHAR(100), 
        tipo_pelicula VARCHAR(50) NOT NULL, 
        anio_pelicula INT NOT NULL, 
        genero_pelicula VARCHAR(20) NOT NULL
    )
    """)
    print("Tabla 'datos_peliculas' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Tabla 'datos_peliculas' creada exitosamente.


- Insertar datos dentro de la tabla 1 = **datos_peliculas**

In [24]:
# Insertar datos dentro de la tabla 1 = movie (corresponde solo al tipo movie dentro de datos_peliculas ya que en los siguientes codigos, solo usamos tipo movie)

sql = "INSERT INTO datos_peliculas (id_pelicula, titulo_pelicula, tipo_pelicula, anio_pelicula, genero_pelicula) VALUES (%s,%s,%s,%s,%s) "
val = movie

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


418 registro/s insertado/s.


In [ ]:
# Obtiene los nombres en la posición 2 (0,1,2) que es en donde está el type
nombres_tipos = set(pelicula [2] for pelicula in datos_peliculas)
print("Los tipos dentro de posición 2 son:", nombres_tipos)

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Movie"]
tv_series = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Series"]
short = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Short"]
tv_episode = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Episode"]
video = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video"]
tv_movie = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "TV Movie"]
video_game = [pelicula for pelicula in datos_peliculas if pelicula[2].replace('_', ' ') == "Video Game"]

print(movie)


- Extracción Selenium datos **peliculas IMDB**

In [25]:

# Iniciar el navegador
driver = webdriver.Chrome()
print("Abriendo Google")
# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")
sleep(3)

# Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")
sleep(3)

# Aceptar cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")
sleep(5)

# Diccionario vacío en dónde se almacenarán los datos
datos_imdb = {"id_pelicula":[], "nombre_pelicula":[], "puntuacion":[], "direccion":[], "guionista":[], "argumento":[] }

# Crear listas vacías para cada tipo de película en minúsculas y añadir info con list comprehension
#tv_mini_series_lista = [pelicula for pelicula in datos_peliculas if pelicula[2] == "TV Mini Series"]
movie = [pelicula for pelicula in datos_peliculas if pelicula[2] == "Movie"]


# Iterar sobre los IDs de las películas
id_pelicula_list = [pelicula[0] for pelicula in movie]

# Iterar sobre los IDs de las películas
for id_pelicula in id_pelicula_list:
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"
    
    # Navegar a la página de la película usando el ID
    driver.get(url_pelicula)
    sleep(5)

    try:
        # Extraer el nombre de la película
        titulo = driver.find_element("css selector", "h1 span").text
        datos_imdb["nombre_pelicula"].append(titulo)
        datos_imdb["id_pelicula"].append(id_pelicula)
        print(f"Titulo: {titulo}")

        # Extraer la puntuación
        try:
            puntuacion = driver.find_element("css selector", "span.sc-eb51e184-1.ljxVSS").text
            # LIMPIEZA
            puntuacion = puntuacion.replace(",", ".")
            puntuacion = float(puntuacion)
            datos_imdb["puntuacion"].append(puntuacion)
            print(f"Puntuacion: {puntuacion}")
        except:
            datos_imdb["puntuacion"].append("N/A")
            print("Puntuación no encontrada")

        # Extraer la dirección
        try:
            directores = driver.find_element("css selector", 'ul > li:nth-child(1) > div').text
            datos_imdb["direccion"].append(directores)
            print(f"Dirección: {directores}")

        except:
                try:
                    directores = driver.find_element("css selector", 'ul > li:nth-child(1) > div').text
                    datos_imdb["direccion"].append(directores)
                    print(f"Dirección: {directores}")
                except:
                    datos_imdb["direccion"].append("N/A")
                    print("Dirección no encontrada")

        # Extraer guionista
        try:
            guionista = driver.find_element("css selector", 'ul > li:nth-child(2) > div').text
            # LIMPIEZA               Los guionistas, asl ser varios y ser extraídos de una caja, los trae juntos por lo que hay que limpiarlos
            guionista = guionista.strip()
            guionista = guionista.replace('\n', ', ').replace(' , ', ', ').strip(', ')
            # Appendeo
            datos_imdb["guionista"].append(guionista)
            print(f"Guionista: {guionista}")
        except:
            try:
                guionista = driver.find_element("css selector", 'ul > li:nth-child(2) > div').text
                datos_imdb["guionista"].append(guionista)
                print(f"Guionista: {guionista}")
            except:
                datos_imdb["guionista"].append("N/A")
                print("Guionista no encontrado")

        # Extraer argumento
        try:
            argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.gxWIhN > div.sc-491663c0-6.etrlfo > div.sc-491663c0-10.iaQXlA > section > p > span.sc-2d37a7c7-2.ggeRnl').text
            datos_imdb["argumento"].append(argumento)
            print(f"Argumento: {argumento}")

           
        except:
            
            try: 
                argumento = driver.find_element("css selector", '#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-4.ILcwq > div.sc-491663c0-6.bvzCJs > div.sc-491663c0-10.tkbDg > section > p > span.sc-2d37a7c7-2.ggeRnl').text
                datos_imdb["argumento"].append(argumento)
                print(f"Argumento: {argumento}")

            except:
                datos_imdb["argumento"].append("N/A")
                print("Argumento no encontrado")

    except:
        print(f"Error al procesar la película con ID '{id_pelicula}'")
        datos_imdb["nombre_pelicula"].append("N/A")
        datos_imdb["id_pelicula"].append(id_pelicula)
        datos_imdb["puntuacion"].append("N/A")
        datos_imdb["direccion"].append("N/A")
        datos_imdb["guionista"].append("N/A")
        datos_imdb["argumento"].append("N/A")

print(datos_imdb)
# Cerrar el navegador
driver.quit()

# Convertir a pandas frame
df_peliculas_imdb = pd.DataFrame(datos_imdb)

# Salvar la busqueda en un documento con formato CSV
df_peliculas_imdb.to_csv("peliculas_imdb.csv", index=False)

print("Data extraction complete and saved to peliculas_imdbs.csv.")

Abriendo Google
Pantalla maximizada
Accediendo a la web
Cookies aceptadas
Titulo: Apaföld
Puntuacion: 5.2
Dirección: Viktor Oszkar Nagy
Guionista: Viktor Oszkar Nagy
Argumento: 
Titulo: Rózsaszín sajt
Puntuacion: 5.9
Dirección: Barnabás Tóth
Guionista: Barnabás TóthFrank RizzoDániel Hamvas
Argumento: To escape the oppressive hyperactivity of his gynecologist father, Dani, 19-year-old student at the Medical University of Budapest, travels to France to his musician uncle. A vision of adolescence and father-son-women relationship filled with humour and human touches. This movie is spoken in Hungarian and French and subtitled in English.
Titulo: Sanam Teri Kasam
Puntuacion: 4.2
Dirección: Lawrence D'Souza
Guionista: Talat Rekhi
Argumento: Se forma un triángulo amoroso cuando un hombre descubre que su mejor amigo se ha casado con su ex prometida, desatando un enredo de amor, traición y emociones encontradas.
Titulo: Twice as Dead
Puntuacion: 7.6
Dirección: Terence H. Winkless
Guionista: Ral

- Convertir a lista de tuplas

In [34]:
# Convertir los datos del diccionario en una lista de tuplas con verificación de valores vacíos
datos_imdb_tuplas = list(zip(
    [id_pelicula if id_pelicula != "" else "N/A" for id_pelicula in datos_imdb["id_pelicula"]],    # IDs de las películas
    [nombre if nombre != "" else "N/A" for nombre in datos_imdb["nombre_pelicula"]],               # Títulos de las películas
    [puntuacion if puntuacion != "" else "N/A" for puntuacion in datos_imdb["puntuacion"]],        # Puntuaciones
    [direccion if direccion != "" else "N/A" for direccion in datos_imdb["direccion"]],            # Directores
    [guionista if guionista != "" else "N/A" for guionista in datos_imdb["guionista"]],            # Guionistas
    [argumento if argumento != "" else "N/A" for argumento in datos_imdb["argumento"]]             # Argumentos
))

# Imprimir la lista de tuplas
print(datos_imdb_tuplas)


[('tt0143558', 'Apaföld', 5.2, 'Viktor Oszkar Nagy', 'Viktor Oszkar Nagy', 'N/A'), ('tt0153140', 'Rózsaszín sajt', 5.9, 'Barnabás Tóth', 'Barnabás TóthFrank RizzoDániel Hamvas', 'To escape the oppressive hyperactivity of his gynecologist father, Dani, 19-year-old student at the Medical University of Budapest, travels to France to his musician uncle. A vision of adolescence and father-son-women relationship filled with humour and human touches. This movie is spoken in Hungarian and French and subtitled in English.'), ('tt0205380', 'Sanam Teri Kasam', 4.2, "Lawrence D'Souza", 'Talat Rekhi', 'Se forma un triángulo amoroso cuando un hombre descubre que su mejor amigo se ha casado con su ex prometida, desatando un enredo de amor, traición y emociones encontradas.'), ('tt0254832', 'Twice as Dead', 7.6, 'Terence H. Winkless', 'Raly RadouloffTerence H. Winkless', 'Jack and Jill rob a bank and end up on the road with their contact and a sexy vixen on the run from the law.'), ('tt0311698', 'Rail

- Crear tabla y columnas tabla 2 (**peliculas imdb**)

In [31]:
# Seleccionar la base de datos
try:
    mycursor.execute("USE BBDD_CinemExtract")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 3 = peliculas_imdb, sus columnas y sus tipos de datos
try:
    mycursor.execute("""
    CREATE TABLE peliculas_imdb (
        id_imdb VARCHAR (15) PRIMARY KEY,
        titulo_imdb VARCHAR(100),
        puntuacion FLOAT DEFAULT NULL, 
        direccion VARCHAR(200) DEFAULT NULL, 
        guionista VARCHAR(200) DEFAULT NULL, 
        argumento VARCHAR(800) DEFAULT NULL
    )
    """)
    print("Tabla 'peliculas_imdb' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Tabla 'peliculas_imdb' creada exitosamente.


- Insertar datos dentro de la tabla 2 = **peliculas_imdb**

In [33]:
# Insertar datos dentro de la tabla 2 = peliculas_imdb

sql = "INSERT INTO peliculas_imdb (id_imdb,titulo_imdb, puntuacion, direccion, guionista, argumento) VALUES (%s,%s,%s,%s,%s,%s) "
val = datos_imdb_tuplas

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)

1265 (01000): Data truncated for column 'puntuacion' at row 32
Error Code: 1265
SQLSTATE 01000
Message Data truncated for column 'puntuacion' at row 32


- Extracción Selenium **actores** de IMDB

In [14]:
# Iniciar el navegador
driver = webdriver.Chrome()
print("Abriendo Google")

# Maximizar la ventana del navegador
driver.maximize_window()
print("Pantalla maximizada")

sleep(3)

# Abrir página web
driver.get("https://www.imdb.com/")
print("Accediendo a la web")

sleep(3)

# Aceptar cookies
driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
print("Cookies aceptadas")

sleep(5)

movie = [pelicula for pelicula in datos_peliculas if pelicula[2] == "Movie"]

# Almacenar resultados
lista_actores = []
datos_actores = {"nombre":[], "anio_nacimiento":[], "conocido_por":[], "que_hace?":[], "premios":[]}
# ----------------------------------------------------------------------------------------------------------------------------------------------------------------------


# Iterar sobre los IDs de las películas
id_pelicula_list = [pelicula[0] for pelicula in movie]
for id_pelicula in id_pelicula_list:
    url_pelicula = f"https://www.imdb.com/title/{id_pelicula}/"

    # Navegar a la página de la película usando el ID
    driver.get(url_pelicula)
    sleep(5)

    # En este caso, el scroll desplazará la página hasta que el eje vertical tenga un valor de 500 píxeles
    driver.execute_script("window.scrollTo(0,800)")
    print('scroll realizado')

    # Pausa de 10 segundos para asegurar que el contenido se cargue completamente
    sleep(10)

    # Extraer los nombres de los actores
    actores_elements = driver.find_elements('css selector', '[data-testid="title-cast-item__actor"]')

    # Crear una lista solo con los nombres de los primeros 10 actores
    actores_list = [actor.text for actor in actores_elements[:10]]
    
    # Añadir los actores al diccionario con el ID de la película como clave 
    datos_actores[id_pelicula] = actores_list 
    datos_actores["nombre"].extend(actores_list )
    print(f'Actores de la película {id_pelicula} agregados: {actores_list}')

    # Que hace
    for actor_element in actores_elements[:10]:
        actor_element.click()
        try:
            que_hace = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-afa4bed1-0.iMxoKo > section > div:nth-child(5) > section > section > div.sc-491663c0-3.kNMkxz > div").text
            datos_actores[id_pelicula] = que_hace 
            print(f'Actor {actor_element} de la película {id_pelicula} agregados: {que_hace}')
        except Exception as e: 
            print("error")

        
    

print(datos_actores)


Abriendo Google
Pantalla maximizada
Accediendo a la web
Cookies aceptadas
scroll realizado
Actores de la película tt0143558 agregados: ['Tamás Ravasz', 'János Derzsi', 'Andrea Nagy', 'Lukács Bicskey', 'István Znamenák']
Actor <selenium.webdriver.remote.webelement.WebElement (session="fe191d4d3f1d1f4d240ce9bd4a148fc8", element="f.32A11C0717DF2A4F6302F1379F47FD81.d.D97F618B9429DD26B104D97CD0A95529.e.256")> de la película tt0143558 agregados: Tamás Ravasz(II)
Reparto


StaleElementReferenceException: Message: stale element reference: stale element not found
  (Session info: chrome=128.0.6613.120); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF7D070B5D2+29090]
	(No symbol) [0x00007FF7D067E689]
	(No symbol) [0x00007FF7D053B1CA]
	(No symbol) [0x00007FF7D054DBD9]
	(No symbol) [0x00007FF7D0542944]
	(No symbol) [0x00007FF7D0540777]
	(No symbol) [0x00007FF7D0544071]
	(No symbol) [0x00007FF7D0544110]
	(No symbol) [0x00007FF7D05910CB]
	(No symbol) [0x00007FF7D0582C16]
	(No symbol) [0x00007FF7D05B66EA]
	(No symbol) [0x00007FF7D05826A6]
	(No symbol) [0x00007FF7D05B6900]
	(No symbol) [0x00007FF7D05D65D9]
	(No symbol) [0x00007FF7D05B6493]
	(No symbol) [0x00007FF7D05809B1]
	(No symbol) [0x00007FF7D0581B11]
	GetHandleVerifier [0x00007FF7D0A28C5D+3295277]
	GetHandleVerifier [0x00007FF7D0A74843+3605523]
	GetHandleVerifier

- Casilla para verificar el output

In [16]:
print(actores_list )

['Tamás Ravasz', 'János Derzsi', 'Andrea Nagy', 'Lukács Bicskey', 'István Znamenák']


- Extracción Beautiful Soup **Premios Oscar**

In [9]:
# URL de la página de Wikipedia
url_oscar = "https://es.wikipedia.org/wiki/Premios_%C3%93scar"

# Realizar la solicitud HTTP a la página
res_url_oscar = requests.get(url_oscar)

# Verificar si la solicitud fue exitosa
if res_url_oscar.status_code == 200:
    # Parsear el contenido HTML con BeautifulSoup
    sopa_oscar = BeautifulSoup(res_url_oscar.content, "html.parser")

    # dict vacío en dónde se almacenarán los datos encontrados
    dict_datos_oscar = {"Ceremonia": [], "Mejor película": [], "Mejor director": [], "Mejor actor": [], "Mejor actriz": []}

    # Extraemos los datos de, únicamente la tabla dada.
    tabla_oscar = sopa_oscar.select_one("#mw-content-text .wikitable")
    # css selector COMPLETO: #mw-content-text > div.mw-content-ltr.mw-parser-output > table.wikitable
    # El selector se ha simplificado en caso de haber tablas anidadas. Y PORQUE CON EL COMPLETO NO NOS SALÍA ...... 
    
    # Crear una lista vacía para almacenar las tuplas
    lista_tuplas_oscar = []

    # Verificar que la tabla haya sido extraída correctamente
    if tabla_oscar:
        # Iterar sobre la tabla buscando las filas (Inspeccionando la web, vemos que las filas están en tr)
        filas = tabla_oscar.find_all('tr')
        
        # Iterando sobre las filas encontradas dentro de la tabla
        for fila in filas[1:]:  # La primera fila es el encabezdo que no necesitamos.
            
            celdas = fila.find_all('td') # Inspeccionando la web, vemos que en td, está el contenido de las celdas
            if len(celdas) >= 6: # Hay que revisar la cantidad de posiciones que tenemos
                
                # Extraer contenido de celdas por posición/ index 
                ceremonia = celdas[1].text.strip() #extraemos por index y sólo el texto y sin espacios extras
                mejor_pelicula = celdas[2].text.strip()
                mejor_director = celdas[3].text.strip()
                mejor_actor = celdas[4].text.strip()
                mejor_actriz = celdas[5].text.strip()
                
                #LIMPIEZA DATOS ---------------------------------------------------
                # Quitar la info extra que se encuentra entre paréntesis
                patron = r"\(.*?\)"
                mejor_pelicula_limpio = re.sub(patron, "", mejor_pelicula).strip()
                mejor_director_limpio = re.sub(patron, "", mejor_director).strip()
                mejor_actor_limpio = re.sub(patron, "", mejor_actor).strip()
                mejor_actriz_limpio = re.sub(patron, "", mejor_actriz).strip()

                # ------------------------------------------------------------------
                
                # Sacar el año de la celda seleccionada en el selector wikitable
                fecha_divida = ceremonia.split() # dividir el texto
                fecha_dividida_lista = list(fecha_divida) #convirtiendo a lista
                for i in fecha_dividida_lista: # iterando por los caracteres
                    numeros = i[-4::]     #trayendo unicamente los ultimos 4 caracter que corresponden al año
                anio = int(numeros)  # conviertiendo esos 4 ultimos caracteres a int para que sea un dato numérico


                # Extrar los datos de las películas desde el año 2000 en adelante
                if anio >= 2000:

                    # Añadir los datos al diccionario
                    dict_datos_oscar["Ceremonia"].append(anio) #es el entero sacado antes
                    dict_datos_oscar["Mejor película"].append(mejor_pelicula_limpio)
                    dict_datos_oscar["Mejor director"].append(mejor_director_limpio)
                    dict_datos_oscar["Mejor actor"].append(mejor_actor_limpio)
                    dict_datos_oscar["Mejor actriz"].append(mejor_actriz_limpio)

                    # Creamos una tupla con los datos extraídos de cada fila de la tabla y se añaden a la lista de tuplas ("lista_tuplas_oscar")
                    tupla = (anio, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz)
                    lista_tuplas_oscar.append(tupla)


        # Convertir el diccionario en un DataFrame de pandas
        df_oscar = pd.DataFrame(dict_datos_oscar)
        # Mostrar el DataFrame
        print(df_oscar)

         # Iterar y printar cada tupla de la lista de tuplas ("lista_tuplas_oscar")
        print("_"*280)
        print("Lista de tuplas:")

        
        print(lista_tuplas_oscar)

    else:
        print("No se ha encontrado la tabla 'wikitable'")
else:
    print(f"Error al acceder a la página: {res_url_oscar.status_code}")





    Ceremonia                                 Mejor película        Mejor director     Mejor actor    Mejor actriz
0        2000                                American Beauty             S. Mendes       K. Spacey        H. Swank
1        2001                                      Gladiator         S. Soderbergh        R. Crowe      J. Roberts
2        2002                               A Beautiful Mind             R. Howard   D. Washington        H. Berry
3        2003                                        Chicago           R. Polanski        A. Brody       N. Kidman
4        2004  The Lord of the Rings: The Return of the King            P. Jackson         S. Penn       C. Theron
5        2005                            Million Dollar Baby           C. Eastwood         J. Foxx        H. Swank
6        2006                                          Crash                A. Lee   P. S. Hoffman  R. Witherspoon
7        2007                                   The Departed           M. Scorse

- Crear tabla y columnas tabla 4 (**premios_oscar**)

In [20]:
# Seleccionar la base de datos
try:
    mycursor.execute("USE BBDD_CinemExtract")
    print("Base de datos seleccionada correctamente.")
except mysql.connector.Error as err:
    print("Error al seleccionar la base de datos.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)
    
# Crear tabla 4 = premios_oscar, sus columnas y sus tipos de datos
try:
    mycursor.execute("""
    CREATE TABLE premios_oscar (
        ceremonia INT, 
        mejor_pelicula VARCHAR(100) PRIMARY KEY, 
        mejor_director VARCHAR(100), 
        mejor_actor VARCHAR(100), 
        mejor_actriz VARCHAR(100) 
    )
    """)
    print("Tabla 'premios_oscar' creada exitosamente.")
except mysql.connector.Error as err:
    print("Se produjo un error al crear la tabla.")
    print("Código de Error:", err.errno)
    print("SQLSTATE:", err.sqlstate)
    print("Mensaje:", err.msg)

Base de datos seleccionada correctamente.
Tabla 'premios_oscar' creada exitosamente.


- Insertar datos dentro de la tabla 1 = **premios_oscar**

In [26]:
# Insertar datos dentro de la tabla 4 = premios_oscar

sql = "INSERT INTO premios_oscar (ceremonia, mejor_pelicula, mejor_director, mejor_actor, mejor_actriz) VALUES (%s,%s,%s,%s,%s) "
val = lista_tuplas_oscar

try: 
    mycursor.executemany(sql, val)
    cnx.commit()
    print(mycursor.rowcount, "registro/s insertado/s.")

except mysql.connector.Error as err:
    print(err)
    print("Error Code:", err.errno)
    print("SQLSTATE", err.sqlstate)
    print("Message", err.msg)


25 registro/s insertado/s.
